In [1]:
import numpy as np
import pandas as pd
from importlib import reload 

import util.performance_metrics as pm
 
reload(pm)

url =  "https://raw.githubusercontent.com/xuda1979/DF/master/sp500_companies/data/constituents.csv"
df_symbols =pd.read_csv('./sp500_companies/data/constituents.csv')

df_symbols.head()
all_symbols = list(df_symbols['Symbol'].unique())
all_symbols.append('SPY')

# Import yfinance
import yfinance as yf  
 
# Get the data for the stock Apple by specifying the stock ticker, start date, and end date
data_all =pd.DataFrame({})
for symbol in all_symbols:
    
    data = yf.download(symbol)#,'2016-01-01','2018-01-01')
    data['Symbol'] = symbol
    if(len(data)>0):
        data_all = pd.concat([data_all,data])
 

vix=yf.download('^VIX')

symbols = list(data_all['Symbol'].unique())

data_all.reset_index(inplace=True)

data_all.head()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HRS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- HCP: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TMK: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- TSS: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%****************

,Date,Open,High,Low,Close,Adj Close,Volume,Symbol
0,1970-01-02,6.851562,6.890625,6.843750,6.851562,1.483333,72000.0,MMM
1,1970-01-05,6.859375,6.898438,6.859375,6.890625,1.491789,446400.0,MMM
2,1970-01-06,6.890625,6.960938,6.882812,6.960938,1.507011,176000.0,MMM
3,1970-01-07,6.960938,7.015625,6.945312,7.000000,1.515469,164800.0,MMM
4,1970-01-08,7.000000,7.109375,6.984375,7.093750,1.535765,304000.0,MMM


In [3]:
training_start_date = '2015-01-15'
training_end_date = '2020-09-30'
mask = data_all['Date'] >= training_start_date
mask &= data_all['Date'] <= training_end_date

data_training= data_all[mask] 

In [20]:
def date_filter(df, start_date, end_date):
    mask = df['Date'] >=  start_date
    mask &= df['Date'] <= end_date
    return df[mask]

# Risk Model

In [931]:
def covariance(df, date, symbols, history):
    df_recent = pd.DataFrame({})
    dates = list(df['Date'].unique())
    dates.sort()
    df_recent['Date']= dates[-(history+1):-1]
    for symbol in symbols:
        mask = df['Symbol'] == symbol
 
        if all([date in list(df[mask]['Date'].unique()) for date in dates[-(history+1):-1]]):
            #print(df[mask]['Close'][-(history+1):-1])
            df_recent[symbol] = df[mask]['Close'].to_list()[-(history+1):-1]
    df_return = pd.DataFrame({})
    df_return['Date'] = df_recent['Date']
    for symbol in df_recent.columns:
        if symbol !='Date':
            #print(symbol,df_recent[symbol].div(df_recent[symbol].shift(1)))
            df_return[symbol] = df_recent[symbol].div(df_recent[symbol].shift(1))
    return df_return[1:].set_index('Date').cov() 
 
            
symbols = ['AAPL', 'MMM']
df_cov = covariance(data_training, list(data_training['Date'].unique())[-1], symbols, 10)
cov=df_cov.to_numpy() 
print(cov)

[[7.85042934e-04 7.44584306e-06]
 [7.44584306e-06 3.88371563e-04]]


In [988]:
def objective(x):
    l = len(x)
    assert l == cov.shape[0]
    return x.dot(cov.dot(x))

def constraint(x):
    return np.atleast_1d(np.sum(np.abs(x)) - 1.5)

 
bounds=[[0, 1], [1, 2]]
res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )
res.x    

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.0006603349124938092
            Iterations: 2
            Function evaluations: 6
            Gradient evaluations: 2


<ipython-input-988-e88c3116b488>:11: OptimizeWarning: Unknown solver options: gtol
  res=optimize.minimize(objective, np.array([0, 0]), constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': True}, bounds=bounds )


array([0.25038093, 1.24961907])

In [1025]:
from scipy.optimize import minimize
def portfolio_optimization(cov, k):        
 
    assert 2*k == cov.shape[0]
    def objective(x):
        return x.dot(cov.dot(x))

    def constraint(x):
        return np.atleast_1d(np.sum(np.abs(x))-1)
    x0 = [1/(2*k)]*k+[-1/(2*k)]*k
    x0 = np.array(x0)
    bounds = []
    for i in range(k):
        bounds.append([1/(4*k),1/k])
    for i in range(k):
        bounds.append([-1/k, -1/(4*k)])
        
    res=optimize.minimize(objective, x0, constraints={"fun": constraint, "type": "ineq"},options={'gtol': 1e-8, 'disp': False}, bounds=bounds )
    s= sum(np.abs(res.x))
    return res.x/s 

# Generate Returns

In [4]:
data_training_return = {}
 
for symbol in symbols:
#     print(symbol)
    mask = data_training['Symbol'] == symbol
    data_training_return[symbol] = pd.DataFrame({})
    data_training_return[symbol]['Date'] = data_training[mask]['Date']
    
    data_training_return[symbol]['close_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Close'].shift(1))-1  
    data_training_return[symbol]['open_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Open'] )-1
    data_training_return[symbol]['close_open_return']  = data_training[mask]['Open'].div(data_training[mask]['Close'].shift(1) )-1
    data_training_return[symbol]['vwap_open_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open'].shift(1)+ \
                                                                                        data_training[mask]['High'].shift(1)+ \
                                                                                        data_training[mask]['Low'].shift(1)+ \
                                                                                        data_training[mask]['Close'].shift(1))-1
    
    data_training_return[symbol]['vwap_close_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open']+ \
                                                                                        data_training[mask]['High']+ \
                                                                                        data_training[mask]['Low']+ \
                                                                                        data_training[mask]['Close'])-1
    
    
    data_training_return[symbol].fillna(0, inplace=True)
    data_training_return[symbol].reset_index(inplace=True,drop=True)
    
   


## Pricing regression model 

In [207]:
#model in sm
import statsmodels.api as sm
import numpy as np
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][ :-testing_length])


from sklearn.linear_model import LinearRegression
X = data_training_return['SPY']['close_close_return'] 
dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

sm_reg_map = {}
for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y =  data_training_return[symbol][mask]['close_close_return'] 

        mask = data_training_return['SPY']['Date'].isin(dates)
        X_here = data_training_return['SPY'][mask]['close_close_return'] 
        X_here = np.expand_dims(X_here, axis=1)
       # print(X_here.shape, y.shape, symbol)
        if X_here.shape[0] == len(dates_X):
            X_here = sm.add_constant(X_here)
           # print(X_here)
            model = sm.RLM(y,X_here)#,M=sm.robust.norms.TrimmedMean())
            results = model.fit()
#             print(results.params[0])
            sm_reg_map[symbol] = results

 


# Mean Reverse Effect at opening

## Regression with mean reverse effcet

In [208]:
#model in sm
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][:-testing_length])
 
X_open_close = data_training_return['SPY']['open_close_return'] 
X_close_open = data_training_return['SPY']['open_close_return'] 
dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

sm_reg_map_reverse = {}
for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y_open_close =  data_training_return[symbol][mask]['open_close_return']
#         y_open_close = np.expand_dims(y_open_close, axis=1)
        y_close_open = data_training_return[symbol][mask]['close_open_return']
        y_close_open = np.expand_dims(y_close_open, axis=1)

        mask = data_training_return['SPY']['Date'].isin(dates)
        X_open_close = data_training_return['SPY'][mask]['open_close_return'] 
        X_open_close = np.expand_dims(X_open_close, axis=1)
        
        X_close_open = data_training_return['SPY'][mask]['close_open_return'] 
        X_close_open = np.expand_dims(X_close_open, axis=1)
        
        if symbol in sm_reg_map:
 
            X_close_open = sm.add_constant(X_close_open)
            pred_close_open = sm_reg_map[symbol].predict(X_close_open)
            pred_close_open = np.expand_dims(pred_close_open, axis=1)
            error_close_open = y_close_open-pred_close_open
  
            X = np.concatenate([X_open_close, error_close_open], axis=1)
 
            if len(X)>0 and X.shape[0] == len(y_open_close):

                X  = sm.add_constant(X)
                model = sm.RLM(y_open_close,X)
                results = model.fit()
                sm_reg_map_reverse[symbol] = results
 
 

# Trade at Open(close-open error as regressor)  

## Filtering reverse coefficient and its t-statistics

In [203]:
symbols_set=set(symbols)

In [365]:

c= 0.0006
k= 1

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
result = pd.DataFrame({'Date': testing_dates, 'Net Profit':0})
result.head()

last_symbol = ['1','1']
not_trade_set = set({'CHK'}) 
buy_symbols = []
sell_symbols= []
#not_trade_set = set()
for index, date in enumerate(testing_dates):
    error_map = {}
    mask = data_training_return['SPY']['Date'] == date     
    X_close_open = data_training_return['SPY'][mask]['close_open_return'].iloc[0]
    for symbol in list(symbols_set-not_trade_set):
         
        if symbol not in last_symbol and symbol in sm_reg_map and symbol in sm_reg_map_reverse and symbol !='SPY'and sm_reg_map_reverse[symbol].params[2] < 0:
#         and (sm_reg_map_reverse[symbol].tvalues[2] > ):
                     #and sm_reg_map_reverse[symbol].params[2] < 0:
 
            try:
 
 
                mask = data_training_return[symbol]['Date'] == date     
                y_close_open = data_training_return[symbol][mask]['close_open_return'].iloc[0]


#                 mask = data_training_return['SPY']['Date'] == date     
#                 X_close_open = data_training_return['SPY'][mask]['close_open_return'].iloc[0]


#                 print(symbol, X_close_open)
                #X_close_open= sm.add_constant(X_close_open)
                X_close_open1 = np.array([[1,X_close_open]])
                pred_close_open = sm_reg_map[symbol].predict(X_close_open1)

                error_close_open = y_close_open-pred_close_open

                X = np.array([[1, 0, error_close_open[0]]])

                #X  = sm.add_constant(X)
                pred = sm_reg_map_reverse[symbol].predict(X)
                #print(X, pred)

                error_map[pred[0]] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
 
    for j in range(k):
        if j< len(error_list):   
            symbol = error_list[j][1]
            last_symbol[0] = symbol
            buy_symbols.append(symbol)
            print(f"buy {symbol}")
            mask = data_training_return[symbol]['Date'] == date
            result.loc[result['Date']==date, 'Net Profit'] -= (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 
            
            symbol = error_list[-j-1][1]
            last_symbol[1] = symbol
            sell_symbols.append(symbol)
            print(f"sell {symbol}")
            mask = data_training_return[symbol]['Date'] == date
            result.loc[result['Date']==date, 'Net Profit'] += (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 

            

pm.Performance(result, c)
 
            

buy EQT
sell HPQ
buy ZBH
sell CBOE
buy CMCSA
sell JBHT
buy TMO
sell TEL
buy HUM
sell PCG
buy FAST
sell ES
buy DHR
sell EXPD
buy UNH
sell JBHT
buy USB
sell CDNS
buy MS
sell TXT
buy ETFC
sell CBOE
buy GT
sell BDX
buy CDNS
sell TRV
buy TMO
sell TXN
buy FFIV
sell EBAY
buy CHTR
sell PFG
buy MSFT
sell PCG
buy SPGI
sell WAT
buy MAT
sell CHRW
buy LKQ
sell FLIR
buy CBOE
sell STX
buy EQT
sell FLIR
buy ZBH
sell BDX
buy HPQ
sell PCG
buy FISV
sell ANSS
buy APD
sell MTD
buy WBA
sell INFO
buy ROK
sell AZO
buy PCG
sell CTAS
buy FISV
sell CSCO
buy DHR
sell CNP
buy PCG
sell JBHT
buy CBOE
sell HD
buy HOLX
sell HSIC
buy DISH
sell ETFC
buy SPGI
sell INTU
buy EBAY
sell COG
buy NWSA
sell HOLX
buy NWS
sell CERN
buy INCY
sell FTI
buy NBL
sell CBOE
buy NWSA
sell ANSS
buy ZBH
sell PKI
buy PDCO
sell FFIV
buy ALXN
sell COO
buy PCG
sell COG
buy AZO
sell MKC
buy SLG
sell HD
buy GE
sell ANSS
buy PPL
sell PBCT
buy CTAS
sell PCG
buy NBL
sell BAX
buy PCG
sell EXPD
buy ECL
sell PCAR
buy CBOE
sell FFIV
buy NWSA
sell ROK
b

In [8]:
#model in sm
import statsmodels.api as sm
import numpy as np
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][:-testing_length])
 
X_open_close = data_training_return['SPY']['open_close_return'] 
X_close_open = data_training_return['SPY']['open_close_return'] 
dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

sm_direct_mp = {}
for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y_open_close =  data_training_return[symbol][mask]['open_close_return']
#         y_open_close = np.expand_dims(y_open_close, axis=1)
        y_close_open = data_training_return[symbol][mask]['close_open_return']
        y_close_open = np.expand_dims(y_close_open, axis=1)
        
        if len(y_close_open) >0:
 
            X = sm.add_constant(y_close_open)
            X  = sm.add_constant(X)
            model = sm.OLS(y_open_close,X)
            results = model.fit()
            sm_direct_mp[symbol] = results

 

In [14]:

c= 0.0006
k= 10

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
result = pd.DataFrame({'Date': testing_dates, 'Net Profit':0})
result.head()

last_symbol = ['1','1']
not_trade_set = set({'CHK'}) 
buy_symbols = []
sell_symbols= []
#not_trade_set = set()
for index, date in enumerate(testing_dates):
    error_map = {}
    mask = data_training_return['SPY']['Date'] == date     
    X_close_open = data_training_return['SPY'][mask]['close_open_return'].iloc[0]
    for symbol in symbols:
         
        if (
            symbol in sm_direct_mp and symbol !='SPY'and 
            sm_direct_mp[symbol].params[1] < 0
        ):
 
 
            try:
 
 
                mask = data_training_return[symbol]['Date'] == date     
                y_close_open = data_training_return[symbol][mask]['close_open_return'].iloc[0]
 

                #X  = sm.add_constant(X)
                pred = sm_direct_mp[symbol].predict([1, y_close_open])
      
                error_map[pred[0]] = symbol
            except:
                pass
    error_list = list(error_map.items())
    error_list.sort()
 
 
 
    for j in range(k):
        if j< len(error_list):   
            symbol = error_list[j][1]

            buy_symbols.append(symbol)

            mask = data_training_return[symbol]['Date'] == date
            result.loc[result['Date']==date, 'Net Profit'] -= (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 
            
            symbol = error_list[-j-1][1]

            sell_symbols.append(symbol)

            mask = data_training_return[symbol]['Date'] == date
            result.loc[result['Date']==date, 'Net Profit'] += (1/(2*k))*data_training_return[symbol][mask]['open_close_return'].to_numpy()[0] 

            

pm.Performance(result, c)
 
            

Daily mean return:  0.0014219780221216445
Sharp ratio:  2.577145356688219


In [226]:
df_params = pd.DataFrame(columns=['symbol','const','x1','x2'])

for symbol  in sm_reg_map_reverse:
    params = sm_reg_map_reverse[symbol].params.to_list()
    print([symbol ]+params)
    #{'symbol': symbol, 'const':params[0],'x1':params[1],'x2':params[2]}
    a_series = pd.Series([symbol ]+params, index = df_params.columns)
    df_params = df_params.append(a_series, ignore_index=True)
    #df_params.append([symbol ]+params, ignore_index=True)

df_params.to_csv('params.csv')

In [193]:
l = len(sm_reg_map_reverse)
c = 0
for key, value in sm_reg_map_reverse.items():
    if value.params[2]<0:
        c += 1
c/l
   

0.7328918322295805

##### Trade at Close(open close error as regressor)  

In [121]:
testing_length = 250
training_dates= set(data_training_return['SPY']['Date'][:-testing_length])


import numpy as np
from sklearn.linear_model import LinearRegression

dates_X = set(data_training_return['SPY']['Date'] )
dates_X = dates_X.intersection(training_dates)

reg_map_vwap_close_reverse = {}
mask = data_training_return['SPY']['Date'].isin(dates)
X1 = data_training_return['SPY'][mask]['close_open_return'][1:].to_numpy()
X1 = np.expand_dims(X1, axis=1)
        
X0 = data_training_return['SPY'][mask]['vwap_close_return'][:-1].to_numpy()
X0 = np.expand_dims(X0, axis=1)

for symbol in symbols:
 
 
    
        dates_y = set(data_training_return[symbol]['Date'] )
        dates = dates_y.intersection(dates_X)
        mask = data_training_return[symbol]['Date'].isin(dates)
        y1 =  data_training_return[symbol][mask]['close_open_return'][1:].to_numpy()
        y1 = np.expand_dims(y1, axis=1)
        y0 = data_training_return[symbol][mask]['vwap_close_return'][:-1].to_numpy()
        y0 = np.expand_dims(y0, axis=1)



        
        if symbol in reg_map:
 
            pred = reg_map[symbol].predict(X0)
            pred = np.expand_dims(pred, axis=1)
            error = y0-pred
           # print(X1.shape, y0.shape, y1.shape, error.shape)
 
            X = np.concatenate([X1, error], axis=1)
         
            #if len(dates_X)>0 and X.shape[0] == len(dates_X):
            reg = LinearRegression().fit(X, y1)
            reg_map_vwap_close_reverse[symbol] = reg
            print(reg.coef_)

# reg.score(X, y)

# reg.coef_

# reg.intercept_
 

[[ 0.87633934 -0.03968184]]
[[1.20186789 0.02018615]]
[[0.89871129 0.11020738]]
[[0.9412648  0.02656301]]
[[1.00942267 0.04549461]]
[[ 1.31440331 -0.02069873]]
[[ 1.13812032 -0.09489471]]
[[1.21766337 0.05103626]]
[[0.79390771 0.03257613]]
[[ 1.87635525 -0.03937646]]
[[0.65856373 0.01670917]]
[[ 1.40667812 -0.04010685]]
[[ 0.92218792 -0.03010474]]
[[1.12514427 0.08901187]]
[[0.93258187 0.06554611]]
[[1.05797977 0.07961114]]
[[ 0.97789008 -0.02376775]]
[[ 1.17862203 -0.03620403]]
[[0.52694646 0.0042412 ]]
[[1.09886982 0.05027293]]
[[ 1.17751209 -0.08241091]]
[[ 0.91498046 -0.02096731]]
[[1.02736076 0.04500753]]
[[0.2194318  0.01877074]]
[[ 0.74568733 -0.01428044]]
[[ 1.1856343  -0.00453171]]
[[1.19146264 0.0154215 ]]
[[ 0.66754668 -0.0844241 ]]
[[1.36654121 0.08950003]]
[[0.23248536 0.0229671 ]]
[[ 1.27155955 -0.03676953]]
[[0.21354501 0.02813489]]
[[ 0.91483252 -0.0396084 ]]
[[ 1.13782097 -0.0202112 ]]
[[0.59833729 0.05206952]]
[[0.26393502 0.01378676]]
[[ 1.27234965 -0.05771015]]
[[0.

[[1.15095105 0.01268818]]
[[0.93003901 0.00979517]]
[[1.12969618e+00 8.20299350e-04]]
[[0.72533604 0.06207415]]
[[0.90331191 0.01196432]]
[[ 1.15813342 -0.04568495]]
[[ 0.92672348 -0.04328592]]
[[0.55101839 0.08598244]]
[[ 1.00349764 -0.051604  ]]
[[ 7.84423395e-01 -5.83530074e-04]]
[[0.71208081 0.07239722]]
[[0.30861491 0.26097271]]
[[ 0.52900286 -0.0888156 ]]
[[0.96461163 0.00937016]]
[[0.17948205 0.03016967]]
[[ 1.33490809 -0.03141734]]
[[1.10648118 0.00404322]]
[[ 1.03077349 -0.01030724]]
[[0.94641129 0.05527681]]
[[0.3283804  0.04140402]]
[[ 1.3488315  -0.04723728]]
[[0.61456743 0.06349997]]
[[ 0.85215624 -0.01185122]]
[[0.70408654 0.02167737]]
[[ 1.4187817 -0.0090648]]
[[0.26181956 0.02246174]]
[[0.36747218 0.06890689]]
[[ 1.03862414 -0.03418474]]
[[ 1.00264592 -0.00247819]]
[[1.44611701e+00 9.31506665e-04]]
[[ 1.14793148 -0.21200067]]
[[1.12790559 0.0061275 ]]
[[0.77604195 0.04010972]]
[[ 1.3408859  -0.03036535]]
[[ 1.31128872 -0.04497727]]
[[0.36932388 0.03850791]]
[[0.50443787

In [129]:
c= 0.0003
result = pd.DataFrame({'Date': testing_dates, 'Net Profit':0})
k= 1

testing_dates = list(data_training_return['SPY']['Date'][-testing_length:])
for index, date in enumerate(testing_dates):
    error_map = {}
    if index+1 <len(testing_dates):
        for symbol in symbols:


            if symbol in reg_map and symbol in reg_map_vwap_close_reverse and symbol !='SPY' and reg_map_vwap_close_reverse[symbol].coef_[0][1]<0:





    #             try:

                mask = data_training_return[symbol]['Date'] == date     
 
 
                y0 = data_training_return[symbol][mask]['vwap_close_return'].to_numpy()
                y0 = np.expand_dims(y0, axis=1)
        
        
                mask = data_training_return['SPY']['Date'] == date     
                X0 = data_training_return['SPY'][mask]['vwap_close_return'] 
                X0 = np.expand_dims(X0, axis=1)

                X1 =  np.zeros([len(X0),1])
                #print(X1.shape, y0.shape)
                if X1.shape == y0.shape:

                    pred0  = reg_map[symbol].predict(X0)
                    pred0  = np.expand_dims(pred0, axis=1)
                    error = y0-pred0
                    #print("pred0 shape", pred0.shape)
                    X = np.concatenate([X1, error], axis=1)

                    pred = reg_map_vwap_close_reverse[symbol].predict(X)
                    #print("pred .shape", pred.shape)

                    error_map[pred[0][0]] = symbol
    #             except:
    #                 pass
        error_list = list(error_map.items())
        error_list.sort()


        

        for j in range(k):
            if j< len(error_list) and index <len(testing_dates) -1:   
                symbol = error_list[j][1]
                mask = data_training_return[symbol]['Date'] == testing_dates[index+1]
                #Here use the previouse date for the return from close to open for conevnience of VIX calculation
                result.loc[result['Date']==date, 'Net Profit'] -= (1/(2*k))*data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] 

                symbol = error_list[-j-1][1]
                mask = data_training_return[symbol]['Date'] == testing_dates[index+1]
                result.loc[result['Date']==date, 'Net Profit'] += (1/(2*k))*data_training_return[symbol][mask]['close_open_return'].to_numpy()[0] 

                

pm.Performance(result, c)
pm.VIX(result, 'Close')

Daily mean return:  0.0017189087581574352
Sharp ratio:  1.670738725561171
[*********************100%***********************]  1 of 1 completed
                            OLS Regression Results                            
Dep. Variable:             Net Profit   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.1601
Date:                Sun, 03 Jan 2021   Prob (F-statistic):              0.689
Time:                        20:50:53   Log-Likelihood:                 675.18
No. Observations:                 250   AIC:                            -1346.
Df Residuals:                     248   BIC:                            -1339.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025 

In [128]:
reg_map_vwap_close_reverse[symbol].coef_[0][1]

-0.03968184474390066

# Random Forests

In [1881]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.datasets import make_regression
# X, y = make_regression(n_features=4, n_informative=2,
#                        random_state=0, shuffle=False)
# regr = RandomForestRegressor(n_estimators=1000,max_depth=100, criterion='mae',random_state=0)
# regr.fit(X, y)

# print(regr.predict([[0, 0, 0, 0]]))

[1.63596469]


Generate training samples

In [11]:
history = 30
testing_length=250
X=[]
y=[]
dates = list(data_training['Date'].unique())
dates.sort()
symbols = set(data_training['Symbol'].unique())
c = 0
for index, date in enumerate(dates[:-testing_length]):
    if c>=1000:
        break
    if index>=history-1:
         
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']<date 
        mask &= data_training['Date']>= dates[index-history+1]
                                                    
        sample_spy = list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
 
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']== date 
    
             
 
        if data_training[mask]['Open'].to_numpy()[0]:
            sample_spy.append(data_training[mask]['Open'].to_numpy()[0])  
            for symbol in symbols:
                c +=1
                if c>=1000:
                    break
                try:
                    sample = []
                    mask = data_training['Symbol'] == symbol
                    mask &= data_training['Date']<date
                    mask &= data_training['Date']>= dates[index-history+1]
                    sample += list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
                    sample +=sample_spy



                    mask = data_training_return[symbol]['Date'] == date

                    r = data_training_return[symbol][mask]['close_open_return'].to_numpy()[0]
                    if sample and r:
                        X.append(sample)
                        y.append(r)

                except:
                    print(symbol, date)
    #                 print(data_training_return[symbol]['Date'] )
    #                 raise
X_train=np.array(X)
y_train=np.array(y)
print(X_train.shape, y_train.shape)

PYPL 2015-02-27T00:00:00.000000000
FTV 2015-02-27T00:00:00.000000000
HPE 2015-02-27T00:00:00.000000000
ARNC 2015-02-27T00:00:00.000000000
PX 2015-02-27T00:00:00.000000000
UA 2015-02-27T00:00:00.000000000
BHF 2015-02-27T00:00:00.000000000
IR 2015-02-27T00:00:00.000000000
CSRA 2015-02-27T00:00:00.000000000
KHC 2015-02-27T00:00:00.000000000
FOXA 2015-02-27T00:00:00.000000000
FOX 2015-02-27T00:00:00.000000000
SCG 2015-02-27T00:00:00.000000000
WRK 2015-02-27T00:00:00.000000000
PYPL 2015-03-02T00:00:00.000000000
FTV 2015-03-02T00:00:00.000000000
HPE 2015-03-02T00:00:00.000000000
ARNC 2015-03-02T00:00:00.000000000
PX 2015-03-02T00:00:00.000000000
UA 2015-03-02T00:00:00.000000000
BHF 2015-03-02T00:00:00.000000000
IR 2015-03-02T00:00:00.000000000
CSRA 2015-03-02T00:00:00.000000000
KHC 2015-03-02T00:00:00.000000000
FOXA 2015-03-02T00:00:00.000000000
FOX 2015-03-02T00:00:00.000000000
SCG 2015-03-02T00:00:00.000000000
WRK 2015-03-02T00:00:00.000000000
PYPL 2015-03-03T00:00:00.000000000
FTV 2015-03

In [ ]:
history = 30
testing_length=250
k=5
X=[]
y=[]
dates = list(data_training['Date'].unique())
dates.sort()
symbols = set(data_training['Symbol'].unique())
result = np.zeros(testing_length)
print(len(dates))
for index, date in enumerate(dates ):
 
    if index>=len(dates)-testing_length:
         
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']<date 
        mask &= data_training['Date']>= dates[index-history+1]
                                                    
        sample_spy = list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
 
        mask = data_training['Symbol'] == 'SPY'
        mask &= data_training['Date']== date 
    
        sample_spy.append(data_training[mask]['Open'].to_numpy()[0]) 
        print(sample_spy)
        symbol_return = {}
        symbol_pred_return = {}
        for symbol in symbols:
            try:
                sample = []
                mask = data_training['Symbol'] == symbol
                mask &= data_training['Date']<date
                mask &= data_training['Date']>= dates[index-history+1]
                sample += list(data_training[mask][['Open','High','Low','Close','Adj Close', 'Volume']].to_numpy().flat)
                sample +=sample_spy
                


                mask = data_training_return[symbol]['Date'] == date
         
                r = data_training_return[symbol][mask]['close_open_return'].to_numpy()[0]
                if sample and r:
                    X.append(sample)
                    symbol_pred_return[symbol] = regr.predict(X)
                    symbol_return[symbol] = r 
                
            except:
                print("Error", symbol, date)
#                 print(data_training_return[symbol]['Date'] )
#                 raise
        error_map = {}
        for symbol in  symbol_return:
            error_map[symbol] = symbol_return[symbol] - symbol_pred_return[symbol]
        error_list = list(error_map.items())
        error_list.sort()
       # print(error_list)
        if len(error_list)>=k:
            for j in range(k): 
                symbol = error_list[j][1]
                print(symbol)
                result[index-len(dates)+testing_length] +=(1/(2*k)*data_training[symbol]['open_close_return']
                symbol = error_list[-j-1][1]
                result[index-len(dates)+testing_length] -=(1/(2*k)*data_training[symbol]['open_close_return']
    X=np.array(X)
y=np.array(y)
print(X.shape, y.shape)
Performance(result, c)
            

# Model deployment

In [373]:
def get_training_data(start_date):
    mask = data_all['Date'] >= start_date
    data_training= data_all[mask]
    return data_training

data_training_de = get_training_data('20170101')

def get_return_data(data_training):
    data_training_return = {}

    for symbol in symbols:
    #     print(symbol)
        mask = data_training['Symbol'] == symbol
        data_training_return[symbol] = pd.DataFrame({})
        data_training_return[symbol]['Date'] = data_training[mask]['Date']

        data_training_return[symbol]['close_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Close'].shift(1))-1  
        data_training_return[symbol]['open_close_return']  = data_training[mask]['Close'].div(data_training[mask]['Open'] )-1
        data_training_return[symbol]['close_open_return']  = data_training[mask]['Open'].div(data_training[mask]['Close'].shift(1) )-1
        data_training_return[symbol]['vwap_open_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open'].shift(1)+ \
                                                                                            data_training[mask]['High'].shift(1)+ \
                                                                                            data_training[mask]['Low'].shift(1)+ \
                                                                                            data_training[mask]['Close'].shift(1))-1

        data_training_return[symbol]['vwap_close_return']  = 4*data_training[mask]['Open'].div(data_training[mask]['Open']+ \
                                                                                            data_training[mask]['High']+ \
                                                                                            data_training[mask]['Low']+ \
                                                                                            data_training[mask]['Close'])-1


        data_training_return[symbol].fillna(0, inplace=True)
        data_training_return[symbol].reset_index(inplace=True,drop=True)
    return data_training_return

data_training_de = get_return_data(data_training_de)


#model in sm
import statsmodels.api as sm
import numpy as np
def train_model(data_training):
 
    training_dates= set(data_training_return['SPY']['Date'])


    from sklearn.linear_model import LinearRegression
    X = data_training_return['SPY']['close_close_return'] 
    dates_X = set(data_training_return['SPY']['Date'] )
    dates_X = dates_X.intersection(training_dates)

    sm_reg_map = {}
    for symbol in symbols:



            dates_y = set(data_training_return[symbol]['Date'] )
            dates = dates_y.intersection(dates_X)
            mask = data_training_return[symbol]['Date'].isin(dates)
            y =  data_training_return[symbol][mask]['close_close_return'] 

            mask = data_training_return['SPY']['Date'].isin(dates)
            X_here = data_training_return['SPY'][mask]['close_close_return'] 
            X_here = np.expand_dims(X_here, axis=1)
           # print(X_here.shape, y.shape, symbol)
            if X_here.shape[0] == len(dates_X):
                X_here = sm.add_constant(X_here)
               # print(X_here)
                model = sm.OLS(y,X_here)#,M=sm.robust.norms.TrimmedMean())
                results = model.fit()
    #             print(results.params[0])
                sm_reg_map[symbol] = results

 

    sm_reg_map_reverse = {}
    for symbol in symbols:



            dates_y = set(data_training_return[symbol]['Date'] )
            dates = dates_y.intersection(dates_X)
            mask = data_training_return[symbol]['Date'].isin(dates)
            y_open_close =  data_training_return[symbol][mask]['open_close_return']
    #         y_open_close = np.expand_dims(y_open_close, axis=1)
            y_close_open = data_training_return[symbol][mask]['close_open_return']
            y_close_open = np.expand_dims(y_close_open, axis=1)

            mask = data_training_return['SPY']['Date'].isin(dates)
            X_open_close = data_training_return['SPY'][mask]['open_close_return'] 
            X_open_close = np.expand_dims(X_open_close, axis=1)

            X_close_open = data_training_return['SPY'][mask]['close_open_return'] 
            X_close_open = np.expand_dims(X_close_open, axis=1)

            if symbol in sm_reg_map:

                X_close_open = sm.add_constant(X_close_open)
                pred_close_open = sm_reg_map[symbol].predict(X_close_open)
                pred_close_open = np.expand_dims(pred_close_open, axis=1)
                error_close_open = y_close_open-pred_close_open

                X = np.concatenate([X_open_close, error_close_open], axis=1)

                if len(X)>0 and X.shape[0] == len(y_open_close):

                    X  = sm.add_constant(X)
                    model = sm.OLS(y_open_close,X)
                    results = model.fit()
                    sm_reg_map_reverse[symbol] = results
    return sm_reg_map, sm_reg_map_reverse

sm_reg_map_de, sm_reg_mp_reverse_de = train_model(data_training_de)

def save0(models):
    df_params = pd.DataFrame(columns=['symbol','const','x1'])

    for symbol  in models:
        params = models[symbol].params.to_list()    
 
 
        a_series = pd.Series([symbol ]+params, index = df_params.columns)
        df_params = df_params.append(a_series, ignore_index=True)
    df_params.to_csv('params0.csv', index=False)

def save(models):
    df_params = pd.DataFrame(columns=['symbol','const','x1','x2'])

    for symbol  in models:
        params = models[symbol].params.to_list()    
        #{'symbol': symbol, 'const':params[0],'x1':params[1],'x2':params[2]}
        if params[2]<0:
            a_series = pd.Series([symbol ]+params, index = df_params.columns)
            df_params = df_params.append(a_series, ignore_index=True)
    df_params.to_csv('params.csv', index=False)

save0(sm_reg_map_de)
save(sm_reg_mp_reverse_de)

df=pd.read_csv('params0.csv')
df

# Save close

,symbol,const,x1
0,MMM,-2.793105e-04,0.915876
1,AOS,2.175853e-04,0.899833
2,ABT,3.341339e-04,0.976538
3,ABBV,3.973506e-05,0.853877
4,ACN,3.159727e-04,1.064565
...,...,...,...
447,YUM,1.591667e-04,0.888290
448,ZBH,-1.308196e-04,0.946650
449,ZION,-1.501804e-04,1.121337
450,ZTS,6.610236e-04,0.962055


In [374]:
def save_close(data_all, date):
    dates=list(data_all.Date.unique())
    dates.sort()
    df_y= data_all[data_all.Date == date].reset_index(drop=True)
    df_y = df_y[['Symbol', 'Close']]
    df_y.columns =  df_y.columns.str.lower()
    df_y.to_csv(f'close_{date}.csv',index=False)

In [375]:
save_close(data_all, '20210120')